# 2 Dimensional mantle convection problem in a square box
We change our focus to the slow creeping motion of Earth’s mantle 
over geological timescales.<br>

## Governing Equations - Strong Formulation
The equations governing mantle convection
are derived from the conservation laws of mass, momentum and energy.
The simplest mathematical formulation assumes incompressibility and
the Boussinesq approximation (McKenzie et al., 1973), under which the
non–dimensional momentum and continuity equations are given by:<br>

$$\nabla \cdot \mathbb{\sigma} + Ra_0 \ T \ \hat{k} = 0,$$
$$\nabla \cdot \vec{u} = 0$$

where $\sigma$ is the stress tensor, $\vec{u}$ is the velocity and T temperature. $\hat{k}$ is the unit vector in the direction opposite to gravity and
$Ra_0$ denotes the Rayleigh number, a dimensionless number that quantifies the vigor of convection:

$$Ra0 = \frac{\rho_0 \alpha \Delta T g d^3}{\mu_0 \kappa}$$

Here, $\rho_0$ denotes reference density, $\alpha$ the thermal expansion 
coefficient, $\Delta T$ the characteristic temperature change across the
domain, $g$ the gravitational acceleration, $d$ the characteristic length,
$\mu_0$ the reference dynamic viscosity and $\kappa$ the thermal
diffusivity. <br>

## Governing Equations - Weak Formulation<br>

For the derivation of the finite element discretisation of Equations above, we start by writing these in their weak form.
We select appropriate function spaces V, W, and Q that contain, respectively, the solution fields for velocity u, pressure p, and
temperature T , and also contain the test functions v, w and q. The weak form is then obtained by multiplying these equations
with the test functions and integrating over the domain $\Omega$.

$$\int_\Omega (\nabla v)\colon \mu \left[ \nabla u + \left( \nabla u \right)^T\right] \ dx 
 - \int_{\Omega} \left( \nabla \cdot v\right)\ p \ dx
 - \int_{\Omega} Ra_0\ T\ v\cdot\hat{k} \ dx = 0 \text{ For all } v\in V$$

## Implementation



In [1]:
from firedrake import *

We next need a mesh: for simple domains such as the unit square, 
Firedrake provides built-in meshing functions. As such, the following code 
defines the mesh, with 40 quadrilateral elements in x and y directions. 

In [ ]:
# Mesh - use a built in meshing function:
mesh = UnitSquareMesh(40, 40, quadrilateral=True)
left, right, bottom, top = 1, 2, 3, 4  # Boundary IDs

We also need function spaces, which is achieved by associating the mesh with 
the relevant finite element: V , W and Q are symbolic variables
representing function spaces. They also contain the function space’s 
computational implementation, recording the association of degrees of freedom
with the mesh and pointing to the finite element basis. <br><br>

Function spaces can be combined in the natural way to create mixed function spaces,
as we do on line 5 of the following code, combining the velocity and pressure
function spaces to form a function space for the mixed Stokes problem, Z.


In [ ]:
# Function spaces:
V = VectorFunctionSpace(mesh, family="CG", degree=2)  # Velocity function space (vector)
W = FunctionSpace(mesh, family="CG", degree=1)  # Pressure function space (scalar)
Q = FunctionSpace(mesh, family="CG", degree=2)  # Temperature function space (scalar)
Z = MixedFunctionSpace([V, W])  # Mixed function space

Test functions, v, w and q are subsequently defined and we also specify functions to hold our solutions: z in the mixed function space, noting that a symbolic representation of the two parts – velocity and pressure – is obtained with
split, and Told and Tnew, required for the Crank-Nicolson scheme used for temporal discretisation in our energy equation, where $T_\theta$  is defined.

In [ ]:
# Test functions and functions to hold solutions:
v, w = TestFunctions(Z)
q = TestFunction(Q)
z = Function(Z)
u, p = split(z)  # Returns symbolic UFL expression for u and p
Told, Tnew = Function(Q, name="OldTemp"), Function(Q, name="NewTemp")
Ttheta = 0.5 * Tnew + 0.5 * Told  # Temporal discretisation through Crank-Nicholson

Mantle convection is an initial condition problem. For our first try, we assume the initial temperature distribution to be prescribed by<br> 

$T(x,y) = (1-y) + 0.05\ cos(\pi x)\ sin(\pi y)$ <br>

In the following code, we first obtain symbolic expressions for coordinates in the physical mesh and subsequently use these to initialize the old temperature field. 

In [ ]:
# Initialise temperature field:
X = SpatialCoordinate(mesh)
Told.interpolate(1.0 - X[1] + 0.05 * cos(pi * X[0]) * sin(pi * X[1]))
Tnew.assign(Told)

### Exercise 6.1 
Visualise Told and show how it looks like.

Next, we initialize Tnew with the values of Told, via the assign
function. Important constants in this problem (Rayleigh Number, Ra; viscosity, μ; thermal diffusivity, κ), in addition to the
constant timestep $\Delta t$ and unit vector $\hat{k}$, are defined next. We note that viscosity could also be a Function, if we
wanted spatial variation.

In [ ]:
# Important constants:
Ra, mu, kappa, delta_t = Constant(1e4), Constant(1.0), Constant(1.0), Constant(1e-6)
k = Constant((0, 1))  # Unit vector (in direction opposite to gravity)

We are now in a position to define the variational problems expressed in Equations (25) and (19). Although in this test case
the problems are linear, we maintain the more general nonlinear residual form FStokes(v, u) = 0 and Fenergy(q, T ) = 0, to allow
for straightforward extension to nonlinear problems below. The symbolic expressions for FStokes and FEnergy in the UFL are
given on lines 33-37: the resemblance to the mathematical formulation is immediately apparent. Integration over the domain is390
indicated by multiplication with dx.
14


In [ ]:
# Stokes equations in UFL form:
stress = 2 * mu * sym(grad(u))
F_stokes = inner(grad(v), stress) * dx - div(v) * p * dx - (dot(v, k) * Ra * Ttheta) * dx
F_stokes += -w * div(u) * dx  # Continuity equation
# Energy equation in UFL form:
F_energy = q * (Tnew - Told) / delta_t * dx + q * dot(u, grad(Ttheta)) * dx + dot(grad(q), kappa * grad(Ttheta)) * dx

In [ ]:

# Set up boundary conditions and deal with nullspaces:
bcvx, bcvy = DirichletBC(Z.sub(0).sub(0), 0, sub_domain=(left, right)), DirichletBC(Z.sub(0).sub(1), 0, sub_domain=(bottom, top))
bctb, bctt = DirichletBC(Q, 1.0, sub_domain=bottom), DirichletBC(Q, 0.0, sub_domain=top)

In [ ]:
p_nullspace = MixedVectorSpaceBasis(Z, [Z.sub(0), VectorSpaceBasis(constant=True)])

In [ ]:
# Initialise output:
output_file = File('output.pvd')  # Create output file
u, p = z.split()
u.rename("Velocity"), p.rename("Pressure")

In [ ]:

# Solver dictionary:
solver_parameters = {
    "mat_type": "aij",
    "snes_type": "ksponly",
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
}


In [ ]:

# Setup problem and solver objects so we can reuse (cache) solver setup
stokes_problem = NonlinearVariationalProblem(F_stokes, z, bcs=[bcvx, bcvy])
stokes_solver = NonlinearVariationalSolver(stokes_problem, solver_parameters=solver_parameters, nullspace=p_nullspace, transpose_nullspace=p_nullspace)
energy_problem = NonlinearVariationalProblem(F_energy, Tnew, bcs=[bctb, bctt])
energy_solver = NonlinearVariationalSolver(energy_problem, solver_parameters=solver_parameters)

In [ ]:
# Timestepping aspects
no_timesteps, target_cfl_no = 2000, 1.0
ref_u = Function(V, name="Reference_Velocity")

In [ ]:

def compute_timestep(u):
    """Return the timestep, using CFL criterion"""
    tstep = (1. / ref_u.interpolate(dot(JacobianInverse(mesh), u)).dat.data.max()) * target_cfl_no
    return tstep

In [ ]:

for timestep in range(0, no_timesteps):
    if timestep > 0:
        delta_t.assign(compute_timestep(u))
    if timestep % 10 == 0:
        output_file.write(u, p, Tnew)
    stokes_solver.solve()
    energy_solver.solve()
    vrms = sqrt(assemble(dot(u, u) * dx)) * sqrt(1./assemble(1.*dx(domain=mesh)))
    nu_top = -1. * assemble(dot(grad(Tnew), FacetNormal(mesh)) * ds(top))
    Told.assign(Tnew)i